# IDS Integrated Data Asset Linkage demo

This script is broadly intended to demonstrate how easily RDMF indexed data can be linked and thus prepared for analysts to use.

In [2]:
# import needed packages (not many!)
from google.cloud import bigquery
import pandas as pd

In [3]:
# initiating client for bigquery
client = bigquery.Client(location="europe-west2")

First, we need to link together the `std` and `georef` versions of the required assets, so that we have all of the data's attributed, along with it's geographic information that will link back to the address/geography indexes. These have internal `guid` columns this can be done with.

Some minor engineering of the data is done at this stage, although most of this has already been done as part of the Integrated Data Asset creation:
* replacing unneeded references to geographies in construction age band

First, let's do this for the Energy Performance Certificates data, which will give us insights into housing energy efficiency:

In [12]:
# EPC georef query
# will also need address_entry_id to pull in address index info
query = ("""

SELECT
  UPPER(lmk_key) AS lmk_key,
  guid,
  UPPER(current_energy_rating) AS current_energy_rating,
  UPPER(potential_energy_rating) AS potential_energy_rating,
  UPPER(REGEXP_REPLACE(construction_age_band, r'England and Wales: ', '')) AS construction_age_band
FROM
  `some_path_georef` t1
JOIN
  `some_path_std` t2
ON
  t1.id = t2.guid

""")

table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.EPC_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

query_job = client.query(
    query,
    location="europe-west2",
    job_config = job_config
)  # API request - starts the query


Next, let's do the same for the Land Registry Price Paid (LRPP) data, which will give us house sale price information:

In [47]:
# joining georef and std LRPP tables
query = ("""

SELECT
  *
FROM
  `ingest_ida_land_registry_price_paid.prices_paid_georef` t1
JOIN
  `ingest_ida_land_registry_price_paid.prices_paid_std` t2
ON
  t1.id = t2.guid

""")
table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.lrpp_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

query_job = client.query(
    query,
    location="europe-west2",
    job_config=job_config
)  # API request - starts the query


query_job.result()

Finally, and this is the bit where the Integrated Data Asset version of these datasets comes in handy, we can link all of these sources together on their `address_entry_id` variables. This is the lookup to the RDMF, specifically the address index, and is a unique identifier in this case.

This has powerful implications on policy forming with future analysis; we have a consistent, simple join between these different datasets and the index, so all analysts will have the same reproducible outputs when joining these data to add extra information to their analysis that wasn't available before.

We also have links to the geography and address indexes while the data stays totally deidentified. The analytical team can use these indexes to join on geography and address information to the data.

In [103]:
# joining OSPOS and (newly joined) LRPP tables
query = ("""

SELECT
  geography_entry_id_lsoa_code,
  geography_entry_id_oa_code,
  geography_entry_id_lad_code,
  geography_entry_id_msoa_code,
  geography_entry_id_country_code,
  geography_entry_id_region_code,
  lsoa_code_hashed,
  oa_code_hashed,
  lad_code_hashed,
  msoa_code_hashed,
  country_code_hashed,
  region_code_hashed,
  class,
  voa_ct_record,
  ruc_code,
  ruc,
  ur2fold,
  ur3fold,
  ur6fold,
  ur8fold,
  building_flat_count,
  private_outdoor_space,
  private_outdoor_space_area,
  t1.address_entry_id_uprn,
  t2.address_entry_id_uprn AS t2_address_entry_id_uprn,
  t2.id,
  apiversion,
  confidencescore,
  epoch,
  underlyingscore,
  t2.geography_entry_id_postcode,
  t2.postcode_hashed,
  t2.guid,
  t2.transaction_uid,
  price,
  date_of_transfer,
  t2.property_type,
  old_new,
  duration,
  locality,
  ppd_category_type,
  derived_year
FROM
  `ons-ids-data-prod.ingest_ida_ordnance_survey_private_outside_space.april_gb_private_outdoor_space_std` AS t1
JOIN
  `ons-ids-analysis-prod.demo_wip_notebook.lrpp_linked` AS t2
ON
  t1.address_entry_id_uprn = t2.address_entry_id_uprn
  
""")

table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

job_config.write_disposition = "WRITE_TRUNCATE"

query_job = client.query(
    query,
    location="europe-west2",
    job_config=job_config
)  # API request - starts the query

query_job.result()

In [ ]:
# joining OSPOS/LRPP linked data with EPC data
# may need to exclude columns before the join:
# EXCEPT (geography_entry_id_postcode, uprn_hashed, postcode_hashed, guid, property_type, address_entry_id_uprn)

query = ("""

SELECT
  *
FROM
  `ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked` t1
JOIN
  `ons-ids-analysis-prod.demo_wip_notebook.EPC_linked` t2
ON
  t1.address_entry_id_uprn = t2.address_entry_id_uprn;
    
""")

table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_epc_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

query_job = client.query(
    query,
    location="europe-west2",
    job_config=job_config
)  # API request - starts the query

query_job.result()

## Non-matches and bias

For cases where records did not match to an RDMF index, for example the address index, their `address_entry_id` will be null. This allows identification of the non-matching records

In [ ]:
# joining georef and std LRPP tables
query = ("""
SELECT 

 address_entry_id_uprn

FROM 
 `ons-ids-analysis-prod.demo_wip_notebook.lrpp_linked`

""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query


df = query_job.to_dataframe()


In [112]:
unmatched  = len(df[df.address_entry_id_uprn.isnull() == True])
matched  = len(df[df.address_entry_id_uprn.isnull() == False])

print("the match rate is:", ((matched-unmatched)/matched)*100)

the match rate is: 99.98994303084024


The analyst can then easily explore the characteristics of the non-linking data. For instance, exploring the trends in matches vs non-matches by region:

In [10]:
# read in geography index lookup, join on geography_entry_region_code

# hashed values - can aggregate on these hashed values to explore bias (i.e. same hashed uprn for same hh)

# match rate by region

26886828

In [ ]:
geography entry id, MSOA code and MSOA label

In [61]:
# querying ospos lrpp linked data
query = ("""
SELECT 

ur2fold, ur3fold, ur6fold, ur8fold

FROM `ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked`
    
""")
table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked'

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df = query_job.to_dataframe()

In [62]:
df[df.ur2fold != '']

,ur2fold,ur3fold,ur6fold,ur8fold
155395,2,3,6,8
530201,2,3,6,8
624295,2,3,6,8
624389,2,2,5,6
784188,1,1,2,2
...,...,...,...,...
22318104,2,3,6,8
22434681,1,1,1,1
22445613,2,3,6,8
22908829,2,2,5,6


In [59]:
df[df.lsoa_code_hashed == '']

,lsoa_code_hashed,oa_code_hashed,lad_code_hashed,msoa_code_hashed,country_code_hashed,region_code_hashed
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
...,...,...,...,...,...,...
25836155,,,,,,
25836156,,,,,,
25836157,,,,,,
25836158,,,,,,


In [ ]:
# exploring LRPP join
query = ("""
SELECT 
COUNT(*) as numRecords

FROM `ingest_ida_land_registry_price_paid.prices_paid_georef` t1
JOIN `ingest_ida_land_registry_price_paid.prices_paid_std` t2

ON t1.id = t2.guid;
""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df  = query_job.to_dataframe()

In [30]:
# all 26889532 records join
df

,numRecords
0,26889532


In [16]:
# exploring LRPP columns, querying where a false condition is true to just return column names
query = ("""
SELECT * 

FROM `ingest_ida_land_registry_price_paid.prices_paid_std`

WHERE 1 = 0
""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df  = query_job.to_dataframe()


In [17]:
df.columns.tolist()

['geography_entry_id_postcode',
 'postcode_hashed',
 'guid',
 'transaction_uid',
 'price',
 'date_of_transfer',
 'property_type',
 'old_new',
 'duration',
 'locality',
 'ppd_category_type',
 'derived_year']